In [126]:
import pandas as pd
import numpy as np

In [127]:
#Make Results DF for Use Later
resultsdf=pd.DataFrame(columns = ['year','anesN'])
resultsdf.set_index('year', inplace=True)

In [135]:
#Read in the ANES Data for Each Year
#ANES1948 religion not broken out beyond prot, cath, jewi, other, none. Not useful
#ANES1952 same
#ANES1956 same
#ANES1958 same
#... (didn't check)
#1970 has... 700360 R'S RELIGION--PROTESTANT/CATHOLIC/JEWISH/OTHER 700361 R'S RELIGIOUS PREFERENCE--SPECIFIC CHURCH and 700362 XHOW OFTEN DOES R GO TO CHURCH 
#...
#1980 HAS Mormon/LDS Data
#No weights in 1980?
print("reading ANES 1980")
anes1980df = pd.read_stata('ANESData/NES1980.dta')
#...
print("reading ANES 2000")
anes2000df = pd.read_stata('ANESData/anes2000TS.dta')
print("reading ANES 2004")
anes2004df = pd.read_stata('ANESData/anes2004TS.dta')
print("reading ANES 2012")
anes2012df = pd.read_stata("ANESData/anes_timeseries_2012.dta")
print("reading ANES 2020")
anes2020df = pd.read_stata('ANESData/anes_timeseries_2020_stata_20210324.dta')

print('finished reading')

reading ANES 1980
reading ANES 2000
reading ANES 2004
reading ANES 2012
reading ANES 2020
finished reading


In [138]:
anes1980df['V800693'].cat.categories

Index(['100. PROTESTANT,NONDE', '101. NON-DENOMINATION',
       '102. COMMUNITY CHURCH', '109. OTH PROTESTANT', '110. PRESBYTARIAN',
       '111. LUTHERAN -EXCEPT', '112. CONGREGATIONAL', '113. EVANGELICAL REFO',
       '114. REFORMED,DUTCH R', '115. UNITED CHURCH OF',
       '116. EPISCOPALIAN,ANG', '120. METHODIST', '121. AFRICAN METHODIS',
       '122. UNITED BRETHREN', '123. BAPTIST -EXCEPT', '124. DISCIPLES OF CHR',
       '125. `CHRISTIAN-', '126. MENNONITE `AMISH', '131. CHURCH OF GOD;HO',
       '132. NAZARENE - FREE', '133. CHURCH OF GOD CH',
       '135. PENTECOSTAL - AS', '136. CHURCH OF CHRIST', '137. SALVATION ARMY',
       '138. PRIMITIVE,FREE W', '139. SEVENTH DAY ADVE',
       '140. SOUTHERN BAPTIST', '141. MO SYNOD LUTHERA',
       '149. OTH FUNDAMENTALI', '150. CHRISTIAN SCIENT',
       '152. LATTER DAY SAINT', '153. UNITARIAN - UNIV',
       '154. JEHOVAH-S WITNES', '155. QUAKERS', '156. UNITY',
       '200. ROMAN CATHOLIC', '300. JEWISH', '710. GREEK ORTHODOX',
    

In [140]:
#Process Data
anesYears=[]
anesdfs={}

print('Processing ANES year 1980')
anesYears.append(1980)
#No Weights? 
anes1980df["weight"]=1
anes1980df.rename(columns={"V800693":"relig"}, inplace=True)
anes1980df.relig = anes1980df.relig.replace('152. LATTER DAY SAINT','LDS')
anesdfs[1980]=anes1980df

print('Processing ANES Year 2000')
anesYears.append(2000)
anes2000df.rename(columns={"V000002":"weight"}, inplace=True)
anes2000df.rename(columns={"V000884":"relig"}, inplace=True)
anes2000df.relig = anes2000df.relig.replace('23. MORMONS','MORMONS')
anes2000df.relig = anes2000df.relig.replace('25. LATTER DAY SAINTS','MORMONS')
anesdfs[2000]=anes2000df

print('Processing ANES Year 2004')
anesYears.append(2004)
anes2004df.rename(columns={"V040101":"weight"}, inplace=True)
anes2004df.rename(columns={"V043231":"relig"}, inplace=True)
anes2004df['relig'] = anes2004df['relig'].str.upper()
anes2004df['relig']= anes2004df['relig'].str.strip()
anesdfs[2004]=anes2004df

print('Processing ANES Year 2012')
anesYears.append(2012)
anes2012df.rename(columns={"weight_full":"weight"}, inplace=True)
anes2012df.rename(columns={"relig_mastersummary":"relig"}, inplace=True)
#Remember, look for '-2. Missing' and remove them from counts
#missingFilter = (anes2012df['relig_mastersummary']=='-2. Missing')
anes2012df.relig = anes2012df.relig.replace('301. Mormons; Latter Day Saints; Community of Christ','MORMONS')
#relig_church ever attend church or religious services
#relig_churchoft how often do you attend [EVERY WEEK, ALMOST EVERY WEEK, ONCE OR TWICE A MONTH, A FEW TIMES A YEAR, or NEVER/ NEVER, A FEW TIMES A YEAR, ONCE OR TWICE A MONTH, ALMOST EVERY WEEK, or EVERY WEEK]
anesdfs[2012]=anes2012df

print('Processing ANES Year 2020')
anesYears.append(2020)
anes2020df.rename(columns={"V200010a":"weight"}, inplace=True)
anes2020df.rename(columns={"V201435":"relig"}, inplace=True)
#answer 4. Latter-Day Saints (LDS)
anes2020df.relig = anes2020df.relig.replace('4. Latter-Day Saints (LDS)','LDS')
anesdfs[2020]=anes2020df




Processing ANES year 1980
Processing ANES Year 2000
Processing ANES Year 2004
Processing ANES Year 2012
Processing ANES Year 2020


In [141]:
yearsToAnalize=anesYears
for y in yearsToAnalize:
    df=anesdfs[y]
    #missingFilter = (anes2012df['relig']=='-2. Missing')
    resultsdf.loc[y,'anesN']=len(df)
    resultsdf.loc[y,'anesWeightTotal']=df['weight'].sum()
    #LDSfilter = (df['relig']=='25. LATTER DAY SAINTS') | (df['relig']=='23. MORMONS') | (df['relig']=='LDS')
    LDSFilter = (df['relig']=='LDS')
    LDSdf=df.loc[LDSFilter]
    MormonFilter = (df['relig']=='MORMONS')
    Mormondf=df.loc[MormonFilter]
    resultsdf.loc[y,'anesLDSN']=len(LDSdf)
    resultsdf.loc[y,'anesLDSWeightTotal']=LDSdf['weight'].sum()
    resultsdf['anesLDSFracN']=resultsdf['anesLDSN']/resultsdf['anesN']
    resultsdf['anesLDSFracWeighted']=resultsdf['anesLDSWeightTotal']/resultsdf['anesWeightTotal']
    resultsdf.loc[y,'anesMormonN']=len(Mormondf)
    resultsdf.loc[y,'anesMormonWeightTotal']=Mormondf['weight'].sum()
    resultsdf['anesMormonFracN']=resultsdf['anesMormonN']/resultsdf['anesN']
    resultsdf['anesMormonFracWeighted']=resultsdf['anesMormonWeightTotal']/resultsdf['anesWeightTotal']
    
resultsdf

,anesN,anesWeightTotal,anesLDSN,anesLDSWeightTotal,anesLDSFracN,anesLDSFracWeighted,anesMormonN,anesMormonWeightTotal,anesMormonFracN,anesMormonFracWeighted
year,,,,,,,,,,
2000,1807,1806.999878,0.0,0.000000,0,0.000000,30.0,31.4174,0.0166021,0.017386
2004,1212,1212.001900,0.0,0.000000,0,0.000000,0.0,0.0000,0,0.000000
2012,5914,5911.353000,0.0,0.000000,0,0.000000,82.0,102.8440,0.0138654,0.017398
2020,8280,8280.000000,113.0,117.344606,0.0136473,0.014172,0.0,0.0000,0,0.000000
1980,1614,1614.000000,23.0,23.000000,0.0142503,0.014250,0.0,0.0000,0,0.000000


In [36]:
#Preview the Data:

"""
cces2008df['V000884'].cat.categories

df=cces2008df
LDSfilter = (df['V000884']=='25. LATTER DAY SAINTS') | (df['V000884']=='23. MORMONS')
LDSdf=df.loc[LDSfilter]
"""
y=2000
df=anes2000df
resultsdf.loc[y,'anesN']=len(df)
resultsdf.loc[y,'anesWeightTotal']=df['V000002'].sum()
LDSfilter = (df['V000884']=='25. LATTER DAY SAINTS') | (df['V000884']=='23. MORMONS')
LDSdf=df.loc[LDSfilter]
resultsdf.loc[y,'anesLDSN']=len(LDSdf)
resultsdf.loc[y,'anesLDSWeightTotal']=LDSdf['V000002'].sum()

resultsdf['anesLDSFracN']=resultsdf['anesLDSN']/resultsdf['anesN']
resultsdf['anesLDSFracWeighted']=resultsdf['anesLDSWeightTotal']/resultsdf['anesWeightTotal']

In [90]:
resultsdf

,anesN,anesWeightTotal,anesLDSN,anesLDSWeightTotal,anesLDSFracN,anesLDSFracWeighted
year,,,,,,
2000,1807,1806.999878,30.0,31.417400,0.0166021,0.017386
2004,1212,1212.001900,21.0,21.294800,0.0173267,0.017570
2020,8280,8280.000000,113.0,117.344606,0.0136473,0.014172
